# R017 : Unreasonably high or low line item. The difference between an item value and its expected value is above a threshold

In [1]:
import psycopg2
from psycopg2 import OperationalError
from psycopg2 import sql
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from scipy.optimize import curve_fit
import seaborn as sns
from IPython.display import HTML
import webbrowser
from collections import defaultdict
import networkx as nx
import csv
import os
from datetime import datetime
from statsmodels.stats.stattools import medcouple 


os.chdir('/home/francisco/MECAD/2º Ano/Estágio/forcera/scripts/scripts_py')



# LIGAÇÃO À BASE DE DADOS
conn = psycopg2.connect(
    host = "contratos-base-gov1.cf87yxnqgph8.eu-central-1.rds.amazonaws.com",
    port = 5432,
    #database = "contratosbasegov",
    user = "contratosbasegov",
    password = "8n9nyeTBFUyCcLJShNrZdPUai2KQkue4")
cur = conn.cursor()

In [29]:
cur.execute('''ROLLBACK;''')

# 1. Análise para situação em que se consideram os 3 primeiros dígitos do CPV

In [2]:
cur = conn.cursor()
cur.execute('''
            SELECT SUBSTRING(contratos_basegov."cpv",1,3) , id, preco_contratual
            FROM contratos_basegov
            WHERE tipo_procedimento = 'Concurso público' AND preco_contratual > 0;
            ''')

data = pd.DataFrame(cur.fetchall())

In [3]:
data = data.rename(columns={0:'cpv', 1:'id', 2:'preco_contratual'})

cpv = (data.iloc[:,0].unique())
cpv = np.delete(cpv, np.where(cpv == ''))

## 1.1 Inserir indicadores estatísticos na tabela <i>precoc_stat</i>

In [4]:
# GUARDAR TODAS AS COMBINAÇÕES DE CPVs COM OS 3 PRIMEIROS DÍGITOS
cur = conn.cursor()
cur.execute('''
            SELECT DISTINCT(SUBSTRING(cpv,1,3)) 
            FROM contratos_basegov;''')
cpv3 = [item[0] for item in cur.fetchall()]

In [5]:
table_name = 'precoc_stat'
column_names = ['cpv', 'preco_total', 'count', 'mean', 'std', 'min', 'q1', 'q2', 'q3', 'max']

#for i in cpv3:
#    cur.execute('''
#                SELECT preco_contratual
#                FROM contratos_basegov
#                WHERE SUBSTRING(cpv,1,3) = %s AND preco_contratual > 0;
#                ''', (i,))
#    
#    result = pd.DataFrame(cur.fetchall())
#
#    if len(result) > 1:
#        row = list((i,result.iloc[:,0].values.sum()))
#        row.extend(list(result.iloc[:, 0].describe().values))
#        
#        insert_query = sql.SQL("INSERT INTO {} ({}) VALUES ({});").format(
#            sql.Identifier(table_name),
#            sql.SQL(',').join(sql.Identifier(name) for name in column_names),
#            sql.SQL(',').join(sql.Literal(value) for value in row)
#        )
#        cur.execute(insert_query)
#
#conn.commit()

## 1.2 Cálculo da constante de Medcouple

In [6]:
cur = conn.cursor()

for i in cpv:

    df = data.loc[data["cpv"] == i, "preco_contratual"].values
    
    if 1 < len(df) < 12000:
    
        const = float(medcouple(df))
        cur.execute('''
                    UPDATE precoc_stat SET medcouple_constant = %s WHERE precoc_stat."cpv" = %s;
                   ''', (const, str(i)))
    else : 
        cur.execute('''
                    UPDATE precoc_stat SET medcouple_constant = COALESCE(medcouple_constant, 0);
                   ''')
        

conn.commit()

## 1.3 Cálculo da *lower_fence* e *upper_fence*

In [5]:
valores_add = list()

cur = conn.cursor()

for i in list(np.sort(cpv)):
    
    cur.execute('''
               SELECT q1, q3, dq, medcouple_constant
               FROM precoc_stat
               WHERE precoc_stat."cpv" = %s;
              ''', (i,))

    q1,q3,iqr,mc =(cur.fetchall()[0])

    
    if mc != 0: 
       lf = q1 - 1.5*np.exp(-4*mc)*iqr
       uf = q3 + 1.5*np.exp(3*mc)*iqr
       valores_add.append([i,lf,uf])

    
    else :
       lf = q1 - 1.5*iqr
       uf = q3 + 1.5*iqr
       valores_add.append([i,lf,uf])

conn.commit()

In [6]:
cur = conn.cursor()

for i in valores_add:
    cpv, lf, uf = i

    cur.execute('''
                UPDATE precoc_stat
                SET lower_fence = %s, upper_fence = %s
                WHERE precoc_stat."cpv" = %s;
                ''', (lf, uf, str(cpv)))

conn.commit()

# 2. Análise para situação em que se consideram os 2 primeiros dígitos do CPV